In [29]:
import logging
import os
import re
import sys
from itertools import chain

import gensim
import pandas as pd
import torch
from bs4 import BeautifulSoup

from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec


from sklearn.model_selection import train_test_split

import pickle

embed_size = 300
max_len = 760

In [30]:
def get_chunk_article(directory):
    df = pd.DataFrame()

    for filename in os.listdir(directory):

        if filename.endswith('.txt'):
            with open(os.path.join(directory, filename), encoding='utf-8') as file:
                content = file.read()
            df[filename] = [content]

    df = df.T
    new_column_names = {
        0: 'article',
    }

    df.rename(columns=new_column_names, inplace=True)
    return df

def txt_to_wordlist(content):
    # 处理成用空格分隔的单词
    words_sep = re.findall(r'\b\w+\b', content)
    processed_content = ' '.join(words_sep)

    # 全部变为小写，并转为列表
    words = processed_content.lower().split()

    return words

def encode_samples(tokenized_samples):
    features = []
    for sample in tokenized_samples:
        feature = []
        for token in sample:
            if token in word_to_idx:
                feature.append(word_to_idx[token])
            else:
                feature.append(0)
        features.append(feature)
    return features


def pad_samples(features, maxlen=max_len, PAD=0):
    padded_features = []
    for feature in features:
        if len(feature) >= maxlen:
            padded_feature = feature[:maxlen]
        else:
            padded_feature = feature
            while len(padded_feature) < maxlen:
                padded_feature.append(PAD)
        padded_features.append(padded_feature)
    return padded_features

In [31]:
# dir_train_po_1 = '2017depression/cleaned-training/cleaned_positive/chunk_1'
# dir_train_ne_1= '2017depression/cleaned-training/cleaned_negative/chunk_1'
# dir_test_1 = '2017depression/cleaned-testing/chunk_1'
#
# trn_po_1 = get_chunk_article(dir_train_po_1)
# trn_ne_1 = get_chunk_article(dir_train_ne_1)
# tst_1 = get_chunk_article(dir_test_1)
#
# trn_po_1['labels'] = 1
# trn_ne_1['labels'] = 0
#
# train = pd.concat([trn_po_1, trn_ne_1])
# test = tst_1
# train


In [53]:

def read_chunk_files(directory, label ):
    dataframes = []
    for i,filename in enumerate(os.listdir(directory)):
        filepath = os.path.join(directory, filename)
        dataframes = get_chunk_article(filepath)
        if label != 2:
            dataframes['labels'] = label
        break
    for i,filename in enumerate(os.listdir(directory)):
        if i == 0:
            continue
        if filename.startswith("chunk_"):
            filepath = os.path.join(directory, filename)
            df = get_chunk_article(filepath)
            if label != 2:
                df['labels'] = label
            dataframes = pd.concat([dataframes,df])


    return dataframes

# 定义文件夹路径
dir_train_po = '2017depression/cleaned-training/cleaned_positive'
dir_train_ne = '2017depression/cleaned-training/cleaned_negative'
dir_test = '2017depression/cleaned-testing'

# 读取正例和负例的chunk文件并添加标签
train_positive = read_chunk_files(dir_train_po, 1)
train_negative = read_chunk_files(dir_train_ne, 0)
test = read_chunk_files(dir_test,2)

# 合并为train文件
train = pd.concat([train_positive, train_negative])

# 输出合并后的train和test文件
print("Train Dataset:")
print(train.head())
print("\nTest Dataset:")
print(test.head())




C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_19440\2395726363.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[filename] = [content]
C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_19440\2395726363.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[filename] = [content]
C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_19440\2395726363.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

Train Dataset:
                                                                   article  \
train_subject1095_1.txt   The last line is pretty much the same as the ...   
train_subject1190_1.txt   I'm sure I'll get flamed for this, but true f...   
train_subject1191_1.txt               Hi Ruben! Happy Birthday! \n\n\n\n\n   
train_subject1199_1.txt   In the US, the Supreme Court has ruled that i...   
train_subject1201_1.txt   lol I do that on a regular basis. ;) \n\n\n\n...   

                         labels  
train_subject1095_1.txt       1  
train_subject1190_1.txt       1  
train_subject1191_1.txt       1  
train_subject1199_1.txt       1  
train_subject1201_1.txt       1  

Test Dataset:
                                                           article
subject1005.txt   "Kill Bill Siren" was awesome. These seriousl...
subject1062.txt   I know that wind chill is used when it is col...
subject1085.txt   Exactly! I don't know how you can get a job a...
subject1103.txt   Don't tell 

C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_19440\2395726363.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[filename] = [content]
C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_19440\2395726363.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[filename] = [content]
C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_19440\2395726363.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

In [54]:
pickle_file = 'TP_FP_ids_list.pkl'
ids_list = pickle.load(open(pickle_file, 'rb'))

In [55]:
len(ids_list)

367

In [56]:
train_TP_FP = [train['article'][i] for i in ids_list]
label_TP_FP = [train['labels'][i] for i in ids_list]

In [70]:
sum(label_TP_FP)

356

In [57]:
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)

logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ''.join(sys.argv))

2024-04-17 08:41:53,860: INFO: running D:\SoftWare\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py-fC:\Users\FlashBlack7\AppData\Roaming\jupyter\runtime\kernel-50cbecf7-4280-4386-8481-1e188385f67b.json


In [58]:
train

,article,labels
train_subject1095_1.txt,The last line is pretty much the same as the ...,1
train_subject1190_1.txt,"I'm sure I'll get flamed for this, but true f...",1
train_subject1191_1.txt,Hi Ruben! Happy Birthday! \n\n\n\n\n,1
train_subject1199_1.txt,"In the US, the Supreme Court has ruled that i...",1
train_subject1201_1.txt,lol I do that on a regular basis. ;) \n\n\n\n...,1
...,...,...
train_subject9940_9.txt,\n\n\n\n\n\n\n\n\n\n We had done some things. ...,0
train_subject9956_9.txt,But killing him themselves will give them a s...,0
train_subject9958_9.txt,No. There isn't. It is not ionizing radiatio...,0
train_subject9972_9.txt,I have the same problem with changing backgro...,0


In [59]:
# 训练数据的样本和标签分开，通过迭代器
clean_train_article, train_labels = [], []
clean_train_article = [txt_to_wordlist(review) for review in train_TP_FP]
clean_test_article = [txt_to_wordlist(review) for review in test['article']]
train_labels = list(label_TP_FP)

In [60]:
# lenth = 0
#
# # for i in range(len(clean_test_article)):
# #     lenth += len(clean_test_article[i])
#
# for i in range(len(clean_train_article)):
#     lenth += len(clean_train_article[i])
# # print(lenth)
# print(lenth/len(clean_train_article))
#
# lenths = []
#
# # for i in range(len(clean_test_article)):
# #     lenths.append(clean_test_article[i])
#
# for i in range(len(clean_train_article)):
#     lenths.append(len(clean_train_article[i]))
# lenths.sort()
# print(lenths[int(len(lenths)/2)])

In [61]:
# 训练数据、标签、测试集比率， random_state啥？
train_article, val_article, train_labels, val_labels = train_test_split(clean_train_article, train_labels,shuffle=True,
                                                                            test_size=0.2, random_state=42)

In [62]:
# *将序列解压成一个个元素，用chain将其再一次解压(解压回的多个列表进行解压)，最后打成集合便于查找
vocab = set(chain(*clean_test_article)) | set(chain(*clean_train_article))
vocab_size = len(vocab)

In [63]:
# 生成单词与索引的键值对
word_to_idx = {word: i + 1 for i, word in enumerate(vocab)}
word_to_idx['<unk>'] = 0
idx_to_word = {i + 1: word for i, word in enumerate(vocab)}
idx_to_word[0] = '<unk>'

In [64]:
train_features = torch.tensor(pad_samples(encode_samples(train_article)))
val_features = torch.tensor(pad_samples(encode_samples(val_article)))
test_features = torch.tensor(pad_samples(encode_samples(clean_test_article)))

In [65]:
# 标签——张量
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

In [45]:
# 加载词向量模型
# input_file = os.path.join(r"F:\CodeFiled\NLP\lib", 'glove.42B.300d.txt')
wvmodel_file =  os.path.join(r"F:\CodeFiled\NLP\lib", 'glove.42B.300d_word2vec.txt')

# glove2word2vec(input_file, wvmodel_file)
wvmodel = KeyedVectors.load_word2vec_format(wvmodel_file, binary=False)


2024-04-17 08:35:05,245: INFO: loading projection weights from F:\CodeFiled\NLP\lib\glove.42B.300d_word2vec.txt
2024-04-17 08:39:23,021: INFO: KeyedVectors lifecycle event {'msg': 'loaded (1917495, 300) matrix of type float32 from F:\\CodeFiled\\NLP\\lib\\glove.42B.300d_word2vec.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2024-04-17T08:39:23.021049', 'gensim': '4.3.0', 'python': '3.9.0 (default, Nov 15 2020, 08:30:55) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'load_word2vec_format'}


In [66]:
# 标签——张量
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)


weight = torch.zeros(vocab_size + 1, embed_size)


# 根据词向量模型中词汇的大小,
# 索引-词向量模型-》对应单词；对应单词-》词库到索引的映射
for i in range(len(wvmodel.index_to_key)):
    try:
        index = word_to_idx[wvmodel.index_to_key[i]]
        # print(i)
    except:
        # 找不到继续
        continue
    # 索引-词向量模型-》对应单词-》单词到索引-》
    # 索引到单词???-》词库到词向量的映射 -》tensor
    # 总之获取词库中所有单词的特征向量
    weight[index, :] = torch.from_numpy(wvmodel.get_vector(
        idx_to_word[word_to_idx[wvmodel.index_to_key[i]]]))



pickle_file = 'depression_glove_42B_300d_TP_FP.pkl'


C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_19440\3724525410.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels)
C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_19440\3724525410.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_labels = torch.tensor(val_labels)


In [67]:
pickle.dump(
        [train_features, train_labels, val_features, val_labels, test_features, weight, word_to_idx, idx_to_word, vocab],
        open(pickle_file, 'wb'))
print('data dumped!')

data dumped!
